<a href="https://colab.research.google.com/github/rczhen/faiss_playground/blob/main/hnsw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hierarchical Navigable Small World (HNSW) graphs

https://www.pinecone.io/learn/series/faiss/hnsw/

In [ ]:
!pip install faiss-cpu

In [6]:
import faiss
import numpy as np

## Dataset

In [9]:
import shutil
import urllib.request as request
from contextlib import closing

# first we download the Sift1M dataset
with closing(request.urlopen('ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz')) as r:
    with open('sift.tar.gz', 'wb') as f:
        shutil.copyfileobj(r, f)

In [10]:
import tarfile

# the download leaves us with a tar.gz file, we unzip it
tar = tarfile.open('sift.tar.gz', "r:gz")
tar.extractall()

In [11]:
# now define a function to read the fvecs file format of Sift1M dataset
def read_fvecs(fp):
    a = np.fromfile(fp, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy().view('float32')

In [26]:
# data we will search through
xb = read_fvecs('./sift/sift_base.fvecs')  # 1M samples
# also get some query vectors to search with
xq = read_fvecs('./sift/sift_query.fvecs')
# # take just one query (there are many in sift_learn.fvecs)
# xq = xq[0].reshape(1, xq.shape[1])

In [27]:
print(xq.shape)
print(xb.shape)
print(type(xq))

(10000, 128)
(1000000, 128)
<class 'numpy.ndarray'>


## HNSW

We can split ANN algorithms into three distinct categories; trees, hashes, and graphs. HNSW slots into the graph category. More specifically, it is a proximity graph, in which two vertices are linked based on their proximity (closer vertices are linked) — often defined in Euclidean distance.

In [3]:
# setup our HNSW parameters
d = 128  # vector size
M = 32   # number of neighbors we add to each vertex on insertion

index = faiss.IndexHNSWFlat(d, M)
print(index.hnsw)

<faiss.swigfaiss_avx2.HNSW; proxy of <Swig Object of type 'faiss::HNSW *' at 0x78c69d113750> >


In Faiss, these two parameters are set automatically in the set_default_probas method, called at index initialization. The `M_max` value is set to `M`, and `M_max0` set to `M*2`

In [4]:
# the HNSW index starts with no levels
index.hnsw.max_level

-1

In [7]:
# and levels (or layers) are empty too
levels = faiss.vector_to_array(index.hnsw.levels)
np.bincount(levels)

array([], dtype=int64)

In [14]:
index.add(xb)

In [15]:
# after adding our data we will find that the level has been set automatically
index.hnsw.max_level

4

In [16]:
# and levels (or layers) are now populated
levels = faiss.vector_to_array(index.hnsw.levels)
np.bincount(levels)

array([     0, 968746,  30276,    951,     26,      1])

We have the number of levels in our graph, 0 -> 4 as described by max_level.
And we have levels, which shows the distribution of vertices on each level from 0 to 4 (ignoring the first 0 value).
We can even find which vector is our entry point:

In [17]:
index.hnsw.entry_point

118295

## HNSW Details

In [18]:
def set_default_probas(M: int, m_L: float):
    nn = 0  # set nearest neighbors count = 0
    cum_nneighbor_per_level = []
    level = 0  # we start at level 0
    assign_probas = []
    while True:
        # calculate probability for current level
        proba = np.exp(-level / m_L) * (1 - np.exp(-1 / m_L))
        # once we reach low prob threshold, we've created enough levels
        if proba < 1e-9: break
        assign_probas.append(proba)
        # neighbors is == M on every level except level 0 where == M*2
        nn += M*2 if level == 0 else M
        cum_nneighbor_per_level.append(nn)
        level += 1
    return assign_probas, cum_nneighbor_per_level

In [19]:
assign_probas, cum_nneighbor_per_level = set_default_probas(
    32, 1/np.log(32)
)
assign_probas, cum_nneighbor_per_level

([0.96875,
  0.030273437499999986,
  0.0009460449218749991,
  2.956390380859371e-05,
  9.23871994018553e-07,
  2.887099981307982e-08],
 [64, 96, 128, 160, 192, 224])

In [20]:
def random_level(assign_probas: list, rng):
    # get random float from 'r'andom 'n'umber 'g'enerator
    f = rng.uniform()
    for level in range(len(assign_probas)):
        # if the random float is less than level probability...
        if f < assign_probas[level]:
            # ... we assert at this level
            return level
        # otherwise subtract level probability and try again
        f -= assign_probas[level]
    # below happens with very low probability
    return len(assign_probas) - 1

In [21]:
chosen_levels = []
rng = np.random.default_rng(12345)
for _ in range(1_000_000):
    chosen_levels.append(random_level(assign_probas, rng))

In [22]:
np.bincount(chosen_levels)

array([968821,  30170,    985,     23,      1])

## HNSW Performance

In [23]:
index = faiss.IndexHNSWFlat(d, M)

In [24]:
# ef_construction value must be set before we construct the index via index.add(xb),
# but ef_search can be set anytime before searching.

ef_search = 32  # depth of layers explored during search
ef_construction = 64  # depth of layers explored during index construction

index.hnsw.efConstruction = ef_construction
index.add(xb)  # build the index
index.hnsw.efSearch = ef_search

In [29]:
%%time
D, I = index.search(xq[:1000], k=1)

CPU times: user 286 ms, sys: 1 ms, total: 287 ms
Wall time: 158 ms


In [32]:
print(D[:5])
print(I[:5])

[[54229.]
 [51187.]
 [30792.]
 [30510.]
 [47367.]]
[[932085]
 [413247]
 [669835]
 [970797]
 [748397]]
